# Add model 3, which is keeping the number of producers and also the number of  movies|

In [1]:
import json
import os
import sys
import argparse
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import factorial
from scipy import stats
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap
from scipy.stats import ks_2samp
from operator import itemgetter
from copy import copy
from random import random
from math import exp


src_dir = os.path.abspath(os.path.join(os.pardir, os.pardir,'src'))
sys.path[0] = src_dir
from parser.support import ROLES, CREDITS
from parser.my_mongo_db_login import DB_LOGIN_INFO
import parser.support as support
import network.network_builder as build
import network.network_generator as generate
import network.shuffler as shuffle

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Reading data

In [3]:
movie_producer_df = generate.open_movie_data()

Loaded IMDb movies producing_gender_percentage
Got all_movies


# Building synthetic networks


* fixed values
    * number of movies per year
    * number of producers per year

* variables
    * number of people per team - producer_num_list *
    * number of movies per producer - occurence_list *
    * size of gaps - gap_list


In [4]:
unlistyfied_producer_df = generate.unlistify(movie_producer_df, 'producers')

In [5]:
#number of movies per producer of the 90s
unlistyfied_producer_df['producer_id'] = unlistyfied_producer_df.producers.apply(lambda x: x[0])

In [6]:
movies_per_producer = unlistyfied_producer_df.groupby('producer_id').count()['_id']

# Model 3

fixed number of movies, fixed number of producers
given number of movies per producer
given size of gaps per producer/movie

## Model 3- 0

fixed number of movies, fixed number of producers

team size round down/up of the mean

In [7]:
movie_list = generate.bootstrap(movies_per_producer, len(movies_per_producer))

In [8]:
total_num_producers = len(list(set([i[0] for i in unlistyfied_producer_df.producers.tolist()])))

## Assign genders

In [9]:
gender_df = support.get_staff_df('producers')[['_id', 'female_count', 'first_movie', 'last_movie', 'gender']]

In [10]:
females = build.generate_gender_seeds(gender_df)

In [11]:
original_producer_list = [i[0] for sublist in movie_producer_df.producers.tolist() for i in sublist]
seeds = [i for i in females if i in original_producer_list]
len(seeds)

1064

In [12]:
gender_df = support.get_staff_df('producers')[['_id', 'female_count', 'first_movie', 'last_movie', 'gender']]

In [13]:
unlistyfied_producer_df['gender'] = unlistyfied_producer_df.apply(generate.assign_gender, args=(gender_df,), axis=1)

In [14]:
movie_per_producer_gender = {}
for g, g_df in unlistyfied_producer_df.groupby('gender'):
    movies_per_producer = g_df.groupby('producer_id').count()['_id'].tolist()
    movie_per_producer_gender[g] = movies_per_producer

In [15]:
movies = []
for p, group in unlistyfied_producer_df.groupby('producer_id'):
    group_sorted = group.sort_values('year')
    diff = group_sorted.year.diff().values
    if len(group_sorted) > 40:
        print(len(group_sorted))
        print(p)
#         print(group_sorted.year.tolist())
        diff_ratio = Counter(diff[~np.isnan(diff)])
        diffs, occur = zip(*diff_ratio.items())
        diffs = np.array(diffs)
        occur = np.array(occur)
        print(diffs)
        print(occur/sum(occur))
        movies.append(group_sorted.producers.tolist())

43
nm0005544
[3. 0. 1.]
[0.02380952 0.83333333 0.14285714]
43
nm0918424
[3. 0. 1.]
[0.02380952 0.83333333 0.14285714]


In [16]:
gap_dict = generate.get_gaps_gender(unlistyfied_producer_df)

###  Generate movies

In [17]:
number_of_producers_per_year = {}
for year, df in unlistyfied_producer_df.groupby('year'):
    producers = list(set([i[0] for i in df.producers.tolist()]))
    number_of_producers_per_year[year] = len(producers)

In [19]:
#generate new producers every round
generated_producers = generate.generate_producers(total_num_producers)
generated_producers_dict = {}
generated_producers_dict['female'] = generated_producers[:len(seeds)]
generated_producers_dict['male'] = generated_producers[len(seeds):]
total_movie_frame = movie_producer_df[['_id', 'producers', 'year', 'producer_num']].copy(deep=True)

total_movie_frame = generate.assign_team_size(total_movie_frame, number_of_producers_per_year)
total_movie_frame['producers'] = [[]]*len(total_movie_frame)
print('generate movies')
total_num_teams = total_movie_frame.producer_num.sum()

/home/staff/junelee/Research-Project/movie_propagation/src/network/network_generator.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['producer_num'] = df.producer_num.apply(team_size, args=(mean_ceil, mean_floor))
/home/staff/junelee/Research-Project/movie_propagation/src/network/network_generator.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['producer_num'] = df.producer_num.apply(team_size, args=(mean_ceil, mean_floor))
/home/staff/junelee/Research-Project/movie_propagation/src/network/network_generator.py:172: Setting

generate movies


/home/staff/junelee/Research-Project/movie_propagation/src/network/network_generator.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['producer_num'] = df.producer_num.apply(team_size, args=(mean_ceil, mean_floor))
/home/staff/junelee/Research-Project/movie_propagation/src/network/network_generator.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['producer_num'] = df.producer_num.apply(team_size, args=(mean_ceil, mean_floor))
/home/staff/junelee/Research-Project/movie_propagation/src/network/network_generator.py:172: Setting

In [21]:
# generate movies
movie_dict = generate.generate_movie_num(total_num_teams, generated_producers_dict, movie_per_producer_gender)

In [22]:
# assign gaps by gender
movie_dict_per_producer, gap_dict_per_producer = generate.assign_gaps(movie_dict, gap_dict)

In [32]:
final_movie_frame

,_id,producers,year,producer_num,availability
22884,tt0099622,"[CF287345, HO495462, VW979520, XL257020]",1990,4,4
22502,tt0100822,"[LG590818, ZE283951, PQ891079, LW192109]",1990,4,4
10973,tt0100935,"[DC285483, VJ042389, LX404401, IH163343]",1990,4,4
22559,tt0099819,"[BH390317, NK930531, UO985250, QQ477700, OS628...",1990,5,5
24509,tt0099796,"[VS997508, FL087853, HM327025, IG688917]",1990,4,4
2827,tt0099371,"[XD632177, ZC511593, MH800529, OV802293]",1990,4,4
10867,tt0099044,"[DA844086, IO876322, TE125242, RG261134, KB107...",1990,5,5
24478,tt0100606,"[FL076390, DX216032, IF033969, YU514664]",1990,4,4
2923,tt0098987,"[OX389072, CW123216, OC413377, ZM364994]",1990,4,4
24465,tt0100036,"[CF287345, QG323183, AA992995, SV916222, PJ597...",1990,5,5


In [27]:
final_movie_frame = generate.distribute_movies(total_movie_frame, gap_dict_per_producer)

In [28]:
unlistyfied_result_df = generate.unlistify(final_movie_frame, 'producers')
unlistyfied_result_df.head()

,_id,producers,year,producer_num,availability
22884,tt0099622,CF287345,1990,4,4
22884,tt0099622,HO495462,1990,4,4
22884,tt0099622,VW979520,1990,4,4
22884,tt0099622,XL257020,1990,4,4
22502,tt0100822,LG590818,1990,4,4


In [29]:
#check if distributions are correct
movies_data = list(unlistyfied_producer_df.groupby('producer_id').count()['_id'].values)
movies_result = list(unlistyfied_result_df.groupby('producers').count()['_id'].values)

In [30]:
from scipy.stats import ks_2samp
ks_2samp(movies_data, movies_result)

Ks_2sampResult(statistic=0.003820771101076814, pvalue=0.999999999997512)

In [34]:
gap_data = generate.get_gaps(unlistyfied_producer_df)
gap_result = generate.get_gaps(unlistyfied_result_df, 'producers')

In [ ]:
ks_2samp(gap_data, gap_result)

In [ ]:
df_distance = shuffle.find_distances(gap_data, gap_result)
max_p, max_d = shuffle.find_max_distance(df_distance)

In [ ]:
df_match = shuffle.find_max_distance_gaps(unlistyfied_result_df, max_p)

In [ ]:
shuffle.shuffle(unlistyfied_result_df, gap_data)

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
bins = np.arange(0, 10,1)
h1 = ax.hist(gap_data, bins=bins, color='red', alpha=0.3, normed=True)
h2 = ax.hist(gap_result, bins=bins, color='blue', alpha=0.3, normed=True)

In [ ]:
max_d, max_p

In [37]:
df_gen = unlistyfied_result_df.copy(deep=True)

In [46]:
df = pd.read_json('/home/projects/movie-network/data/synthetic_data/model_1_0/movies_1_0_004195.json', orient='split')
df

,_id,producers,year,producer_num
22884,tt0099622,"[FN729157, PW133515, SO061748, LP679122]",1990,4
22502,tt0100822,"[IE968706, KH097645, NO650813, LC449136, XK732...",1990,5
10973,tt0100935,"[MX973798, EW076199, RC688538, EB908473]",1990,4
22559,tt0099819,"[DC465076, UP456489, GH059232, KH017069, VS406...",1990,5
24509,tt0099796,"[TZ434900, PA771212, KX077410, SF431485]",1990,4
2827,tt0099371,"[GM594394, QY556662, WL490712, KC569651]",1990,4
10867,tt0099044,"[CW151304, TZ100116, UD968163, SP756510, DC174...",1990,5
24478,tt0100606,"[TF328013, EJ268580, FZ048392, SB469912]",1990,4
2923,tt0098987,"[LS640297, PS991420, XR931773, IV748544, GO336...",1990,5
24465,tt0100036,"[ZE330977, IN979135, DV699251, BR132369]",1990,4


In [48]:
unlistyfied_result_df[['_id', 'producers', 'year', 'producer_num']].groupby(['_id', 'year', 'producer_num'])['producers'].apply(list)

_id        year  producer_num
tt0081145  1993  4                        [JQ641081, BE679178, ES033591, MP917727]
tt0097115  1990  4                        [XP689574, VS997508, NA296900, SX134279]
tt0097863  1990  5               [UX347801, OH671020, YK668324, FL307371, AS042...
tt0097994  1990  5               [QR296257, KX941011, VE061365, YH085370, JR720...
tt0098966  1990  5               [BH390317, LX283235, SL261177, TV822224, DU114...
tt0098987  1990  4                        [OX389072, CW123216, OC413377, ZM364994]
tt0098994  1990  4                        [XD632177, RR842207, ZF804885, LB053597]
tt0099005  1990  4                        [AP255745, AV908709, IV031471, ZF130303]
tt0099012  1990  4                        [MM839774, BW781766, NQ224175, FQ279569]
tt0099017  1990  4                        [RK072517, YV898062, VZ702528, UV992528]
tt0099018  1990  5               [PR730038, DE809054, SN547602, GR167473, PY545...
tt0099039  1990  4                        [NE930850, LA08

In [45]:
unlistyfied_result_df.columns

Index(['_id', 'producers', 'year', 'producer_num', 'availability'], dtype='object')

In [75]:
def aggregate(df, key_col='_id', val_col='producers'):
    keys, values = df[[key_col, val_col]].sort_values(key_col).values.T
    ukeys, index = np.unique(keys,True)
    arrays = np.split(values,index[1:])
    df2 = pd.DataFrame({key_col:ukeys,val_col:[list(a) for a in arrays]})
    return df2

In [80]:
def organize_dataframe(df_input, key_col='_id', val_col='producers'):
    """
    when the movie-producer unlistyfied result is finalized, put it together into format that is read by the model
    Input:
        df_input - unorgazined dataframe
        key_col - 
        val_col - 
    Output:
        df_output - aggregated dataframe
    """
    df1 = aggregate(df_input, key_col, val_col)
    df2 = df_input[['_id', 'year', 'producer_num']].drop_duplicates()
    
    df_output = pd.merge(df1, df2, on=key_col).sort_values('year').dropna()
    return df_output

In [84]:
df_output = organize_dataframe(unlistyfied_result_df)

In [100]:
df_output

,_id,producers,year,producer_num
134,tt0100486,"[HM456516, SO410050, YY655302, MP917727, WR288...",1990,5
119,tt0100260,"[OW377752, TB132641, CJ213560, QV911417, KS823...",1990,5
120,tt0100287,"[PE734955, XP689574, GY979276, SG077494]",1990,4
121,tt0100318,"[AZ887741, PE734955, AE564250, WW364226]",1990,4
122,tt0100332,"[VH399687, JP086793, QV162017, UF609693]",1990,4
124,tt0100395,"[UR579013, QU070578, YV898062, FK339547, LQ208...",1990,5
125,tt0100403,"[XO736339, BJ387163, JI930594, GL211072, SG674...",1990,5
126,tt0100404,"[WY327610, SL261177, YL761770, ZZ657743]",1990,4
127,tt0100418,"[WN254312, ZC153507, UR579013, UL355351]",1990,4
128,tt0100419,"[WT735340, VF262085, GN140821, AK721588]",1990,4


In [92]:
num_schedules = 10
v = 0
# data_dir = '/home/projects/movie-network/data/synthetic_data/model_3_0/'
data_dir = './'
# gender_dir = '/home/projects/movie-network/data/synthetic_data/genders/'
gender_dir = './'
version_list = generate.make_version(num_schedules)

df_output.to_json(os.path.join(data_dir, 'movies_3_0_{}.json'.format(version_list[v])), orient='split')
#save gender
generated_gender_df = pd.DataFrame(columns=['producer_id', 'gender'])
for g, producers in generated_producers_dict.items():
    genders = [g for ii in producers]
    appending_df = pd.DataFrame({'producer_id': producers, 'gender': genders})
    generated_gender_df = generated_gender_df.append(appending_df)
generated_gender_df.to_json(os.path.join(gender_dir, 'movies_3_0', 'version_{}.json'.format(version_list[v])), orient='split')

In [101]:
hela = pd.read_json('/home/projects/movie-network/data/synthetic_data/model_3_0/movies_3_0_790295.json', orient='split')
hela

,_id,producers,year,producer_num
134,tt0100486,"[NO990264, YX045212, LV212426, TW476447]",1990,4
119,tt0100260,"[UQ418980, KS657678, II533191, DI603597, TZ705...",1990,5
120,tt0100287,"[AH318779, KB845996, SS538288, NO037895, XL066...",1990,5
121,tt0100318,"[TN173391, OX859888, XO039095, KM962422]",1990,4
122,tt0100332,"[RX265529, CG280037, GE623510, JH799916]",1990,4
124,tt0100395,"[NC684567, JR127767, VS826081, XC324243, SM991...",1990,5
125,tt0100403,"[SI261431, BB222094, EQ019501, AI836401]",1990,4
126,tt0100404,"[SU523940, XZ078582, WX191453, PE169645]",1990,4
127,tt0100418,"[CL594236, AZ424087, UD155921, TW998738, VT718...",1990,5
128,tt0100419,"[VA306039, XG140450, AB255264, VA828507, JD766...",1990,5


In [102]:
pd.read_json('/home/projects/movie-network/data/synthetic_data/genders/model_3_0/version_790295.json', orient='split')

,producer_id,gender
0,EM004008,female
1,LX274093,female
2,BB797997,female
3,ZN735801,female
4,FE967799,female
5,OA981971,female
6,IY017688,female
7,MZ250688,female
8,WX858758,female
9,TG699027,female


### Make dataframe with the exact team sizes

In [ ]:
data_dir = '/home/projects/movie-network/data/synthetic_data/model_3_0/'
gender_dir = '/home/projects/movie-network/data/synthetic_data/genders/'

In [ ]:
num_schedules = 1
version_list = net_gen.make_version(num_schedules)

In [ ]:
for key,value in sorted(gap_dict_per_producer.items(),key=lambda i:(sum(i[1]), len(i[1])),reverse=True):
    print(key, value, sum(value))

In [ ]:
total_movie_frame.head()

In [ ]:
shift_dist = []
# for v in range(num_schedules):


#generate new producers every round
generated_producers = net_gen.generate_producers(total_num_producers)
generated_producers_dict = {}
generated_producers_dict['female'] = generated_producers[:len(seeds)]
generated_producers_dict['male'] = generated_producers[len(seeds):]
total_movie_frame = movie_producer_df[['_id', 'producers', 'year', 'producer_num']].copy(deep=True)
print('iterate years')
for year, df in total_movie_frame.groupby('year'):
    num_producers = number_of_producers_per_year[year] #duplicate producers are already dropped
    mean_size = np.mean(df.producer_num.tolist())
    mean_ceil = np.ceil(mean_size)
    mean_floor = np.floor(mean_size)
    #fix the team size to its mean
    df['producer_num'] = df.producer_num.apply(team_size, args=(mean_ceil, mean_floor))
    total_movie_frame['producer_num'].update(df.producer_num)
total_movie_frame['producers'] = np.nan
print('generate movies')
total_num_teams = total_movie_frame.producer_num.sum()
dict_movies = generate_movie_num(total_num_teams, generated_producers_dict, movie_per_producer_gender)
shift_dist.append(list(dict_movies.values()))
print('distribute movies')


    
for i, row in total_movie_frame.iterrows():
    producers, occurence = zip(*dict_movies.items())
    producers = np.array(producers)
    occurence = np.array(occurence)
    size = row.producer_num
    team = np.random.choice(producers, size, replace=False, p=occurence/sum(occurence))
    #assign list to cell
    total_movie_frame['producers'] = total_movie_frame['producers'].astype(object)
    total_movie_frame.at[i, 'producers'] = team
    for p in team:
        dict_movies[p] -= 1
        if dict_movies[p] == 0:
            del dict_movies[p]
            
            
            
#     total_movie_frame.to_json(os.path.join(data_dir, 'movies_3_0_{}.json'.format(version_list[v])), orient='split')
#     #save gender
#     generated_gender_df = pd.DataFrame(columns=['producer_id', 'gender'])
#     for g, producers in generated_producers_dict.items():
#         genders = [g for ii in producers]
#         appending_df = pd.DataFrame({'producer_id': producers, 'gender': genders})
#         generated_gender_df = generated_gender_df.append(appending_df)
#     generated_gender_df.to_json(os.path.join(gender_dir, 'movies_3_0', 'version_{}.json'.format(version_list[v])), orient='split')

In [ ]:
p_list = list(set([i for sublist in total_movie_frame.producers.tolist() for i in sublist]))

In [ ]:
for l in shift_dist:
    print(ks_2samp(l, movie_per_producer_gender['male']+movie_per_producer_gender['female']))

# Model 2-1

fixed number of movies, fixed number of producers

team size sampled from the real schedule

## Fill in producers

In [ ]:
num_schedules = 10

version_list = []
while len(version_list) < num_schedules:
    ver = make_version()
    if ver not in version_list:
        version_list.append(ver)

In [ ]:
from os import listdir
from os.path import isfile, join

original_file_list = [join(original_dir, f) for f in listdir(original_dir) 
                       if isfile(join(original_dir, f))]

## Concat over multiple years

In [ ]:
movie_producer_df[['_id', 'producers', 'year', 'producer_num']].copy(deep=True).head()

In [ ]:
data_dir = '/home/projects/movie-network/data/synthetic_data/model_2_1/'

shift_dist = []
for v in range(num_schedules):
    #generate new producers every round
    generated_producers = net_gen.generate_producers(total_num_producers)
    generated_producers_dict = {}
    generated_producers_dict['female'] = generated_producers[:len(seeds)]
    generated_producers_dict['male'] = generated_producers[len(seeds):]
    total_movie_frame = movie_producer_df[['_id', 'producers', 'year', 'producer_num']].copy(deep=True)
    total_movie_frame['producers'] = np.nan
    print('generate movies')
    total_num_teams = total_movie_frame.producer_num.sum()
    dict_movies = generate_movie_num(total_num_teams, generated_producers_dict, movie_per_producer_gender)
    shift_dist.append(list(dict_movies.values()))
    print('distribute movies')
    for i, row in total_movie_frame.iterrows():
        producers, occurence = zip(*dict_movies.items())
        producers = np.array(producers)
        occurence = np.array(occurence)
        size = row.producer_num
        team = np.random.choice(producers, size, replace=False, p=occurence/sum(occurence))
        total_movie_frame['producers'] = total_movie_frame['producers'].astype(object)
        total_movie_frame.at[i, 'producers'] = team
        for p in team:
            dict_movies[p] -= 1
            if dict_movies[p] == 0:
                del dict_movies[p]
    total_movie_frame.to_json(os.path.join(data_dir, 'movies_2_1_{}.json'.format(version_list[v])), orient='split')
    #save gender
    generated_gender_df = pd.DataFrame(columns=['producer_id', 'gender'])
    for g, producers in generated_producers_dict.items():
        genders = [g for ii in producers]
        appending_df = pd.DataFrame({'producer_id': producers, 'gender': genders})
        generated_gender_df = generated_gender_df.append(appending_df)
    generated_gender_df.to_json(os.path.join(gender_dir, 'movies_2_1', 'version_{}.json'.format(version_list[v])), orient='split')

In [ ]:
for l in shift_dist:
    print(ks_2samp(l, movie_per_producer_gender['male']+movie_per_producer_gender['female']))

# Problem

Random sampling does not account for all of the producers

In [ ]:
ver

In [ ]:
from os.path import normpath, basename, join, isfile
from os import listdir

In [ ]:
network_model =  basename(normpath(data_dir))
gender_folder = os.path.join('/home/projects/movie-network/data/synthetic_data/genders/', network_model)

In [ ]:
gender_file = [join(gender_folder, f) for f in listdir(gender_folder) if isfile(join(gender_folder, f)) and ver in f]

In [ ]:
if len(gender_file) == 1:
    gender_file = gender_file[0]
else:
    raise IndexError('the version has duplicate or it does not exist')

In [ ]:
df_gender = pd.read_json(gender_file, orient='split')
seeds = df_gender[df_gender.gender=='female'].producer_id.tolist()

In [ ]:
len(seeds)

In [ ]:
from os.path import dirname, realpath
two_up = dirname(dirname(__file__))

In [ ]:
os.path.abspath(os.path.join(data_dir, os.pardir, 'gender'))

In [ ]:
data_dir